In [15]:
import json
import argparse
import glob
import sys
sys.path.append("../")
from tool import finqa_equal
from typing import Dict, Any


In [5]:
name = 'convfinqa_direct_gpt3_04_10_07_53.jsonl'

In [7]:
exception = []
correct, wrong, missing = 0, 0, 0
with open(name) as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        if 'demonstration' in data or 'prompt' in data:
            continue
        else:
            # Select the prediction field.
            if 'executed' in data:
                prediction = data['executed']
            else:
                prediction = data['prediction']
                if False:
                    # For multi-choise question, pick A option
                    if prediction == 'None':
                        prediction = 'A'
            # Calculate the evaluation score
            pred = finqa_equal(prediction, data['answer'], True, True)
            if prediction is None:
                missing += 1
            elif pred:
                correct += 1
            else:
                exception.append(data)
                wrong += 1

print('file name: ', name)
print('num of examples: ', correct + wrong + missing)
print('num of missing returns', missing)
print('accuracy: ', correct / (correct + wrong + missing))

file name:  convfinqa_direct_gpt3_04_10_07_53.jsonl
num of examples:  421
num of missing returns 0
accuracy:  0.6247030878859857


In [9]:
import pandas as pd

In [10]:
incorrect = pd.DataFrame(exception)

In [13]:
prompt_4shot = """Read the following text and table, and then answer the last question in a series of questions:
- | shares available for awards | shares subject to outstanding awards
2009 global incentive plan | 2322450 | 2530454
2004 stock incentive plan | - | 5923147
Questions: how many shares are subject to outstanding awards is under the 2009 global incentive plan? what about under the 2004 stock incentive plan? how many total shares are subject to outstanding awards? what about under the 2004 stock incentive plan?
Question: what proportion does this represent?
#Python
shares_subject_to_outstanding_awards_2009_global_incentive_plan = 2530454
shares_subject_to_outstanding_awards_2004_stock_incentive_plan = 5923147
total_shares_subject_to_outstanding_awards = shares_subject_to_outstanding_awards_2009_global_incentive_plan + shares_subject_to_outstanding_awards_2004_stock_incentive_plan
proportion = shares_subject_to_outstanding_awards_2009_global_incentive_plan / total_shares_subject_to_outstanding_awards
ans = proportion


Read the following text and table, and then answer the last question in a series of questions:
compensation expense the company recorded $ 43 million , $ 34 million , and $ 44 million of expense related to stock awards for the years ended december 31 , 2015 , 2014 , and 2013 , respectively . 
Questions: what is the compensation expense the company recorded in 2015? what about in 2014? what is the total compensation expense the company recorded in 2015 and 2014? what is the total expenses including 2013?
Question: what is the average for three years?
#Python
compensation_expense_2015 = 43
compensation_expense_2014 = 34
compensation_expense_2013 = 44
total_compensation_expense = compensation_expense_2015 + compensation_expense_2014 + compensation_expense_2013
average_for_three_years = total_compensation_expense / 3
ans = average_for_three_years


Read the following text and table, and then answer the last question in a series of questions:
the net loss on disposal of those assets was $ 344000 for 2005 and $ 43000 for 2004 . 
Questions: what was the net loss on disposal of assets in 2005? what was the value in 2004? what was the change in value?
Question: what was the percent change?
#Python
net_loss_on_disposal_of_assets_2005 = 344000
net_loss_on_disposal_of_assets_2004 = 43000
net_change_in_value = net_loss_on_disposal_of_assets_2005 - net_loss_on_disposal_of_assets_2004
percent_change = net_change_in_value / net_loss_on_disposal_of_assets_2004
ans = percent_change


Read the following text and table, and then answer the last question in a series of questions:
location | operations conducted | approximatesquare feet | leaseexpirationdates
dublin ireland | global supply chain distribution and administration offices | 160000 | owned
athlone ireland | commercial research and development manufacturing | 80000 | owned
bogart georgia | commercial research and development manufacturing | 70000 | owned
smithfield rhode island | commercial research and development manufacturing | 67000 | owned
Questions: what is the square feet of the owned global supply chain distribution and administration offices? what is the square feet of the owned commercial research and development manufacturing? what is the sum of those values? what is the total sum including square feet of commercial research and development manufacturing in bogart, georgia? what is the total sum including square feet of commercial research and development manufacturing in smithfield, rhode island?
Question: what is the total sum of square feet owned?
#Python
square_feet_owned = [160000, 80000, 70000, 67000]
total_square_feet_owned = sum(square_feet_owned)
ans = total_square_feet_owned
"""

In [16]:
def create_reader_request_processed(example: Dict[str, Any]):
    prompt = 'Read the following text and table, and then answer the last question in a series of questions:\n'
    if example['golden_text']:
        prompt += example['golden_text'].strip() + '\n'
    if example['golden_table']:
        prompt += example['golden_table'].strip() + '\n'
    #prompt += '\n'
    prompt += 'Questions: '
    prompt += " ".join(example['questions'][:-1])
    prompt += '\n'
    prompt += f'Question: {example["questions"][-1]}'
    prompt += '\n'
    prompt += 'Answer:'
    return prompt

In [20]:
full_prompt = prompt_4shot + "\n\n"+ create_reader_request_processed(incorrect.iloc[1,])

In [23]:
incorrect['prompt'] = incorrect.apply(lambda x:  prompt_4shot + "\n\n"+ create_reader_request_processed(x), axis=1)

In [25]:
incorrect

,questions,answer,text,table,golden_text,golden_table,id,programs,generated,executed,prompt
0,[what was the change in the unamortized debt i...,0.375,"as of december 31 , 2017 , the company had gro...",( $ in millions ) | december 31 2017 | decembe...,the company had unamortized debt issuance cost...,( $ in millions ) | december 31 2017 | decembe...,Double_HII/2017/page_104.pdf,"[ans = 15 - 19, x0 = 15 - 19; ans = x0 / 19, a...",[\n#Python\nunamortized_debt_issuance_costs_se...,-0.210526,"Read the following text and table, and then an..."
1,[how much did the gallons hedged in 2018 repre...,14.0,"republic services , inc . notes to consolidate...",year | gallons hedged | weighted average contr...,the aggregate fair values of our outstanding f...,year | gallons hedged | weighted average contr...,Double_RSG/2016/page_144.pdf,"[ans = 12000000 / 3000000, ans = 37.8, ans = 2...",[\n#Python\ngallons_hedged_2017 = 12000000\nga...,0.928571,"Read the following text and table, and then an..."
2,[what is the amount of oil and gas mmboe from ...,24.69136,the acquisition date is on or after the beginn...,- | oil ( mmbbls ) | gas ( bcf ) | ngls ( mmbb...,oil gas ngls total ( mmbbls ) ( bcf ) ( mmbbls...,- | oil ( mmbbls ) | gas ( bcf ) | ngls ( mmbb...,Single_DVN/2007/page_58.pdf-3,"[ans = 60 / 243, x0 = 60 / 243; ans = x0 * 100]",[\n#Python\noil_canada = 23\ngas_canada = 198\...,10.745009,"Read the following text and table, and then an..."
3,[what is the change in price of the s&p 500 fr...,18770.0,"market for registrant 2019s common equity , re...",- | 12/28/2013 | 1/3/2015 | 1/2/2016 | 12/31/2...,,- | 12/28/2013 | 1/3/2015 | 1/2/2016 | 12/31/2...,Single_CDNS/2018/page_31.pdf-1,"[ans = 129.05 - 110.28, x0 = 129.05 - 110.28; ...",[\n#Python\nchange_in_price_2015_2016 = 110.28...,740.0,"Read the following text and table, and then an..."
4,"[what was, in millions, the operating income i...",6.95522,net revenues include $ 3.8 billion in 2017 and...,( in millions ) | for the years ended december...,net earnings attributable to pmi of $ 6.0 bill...,( in millions ) | for the years ended december...,Double_PM/2017/page_38.pdf,"[ans = 11503, ans = 10815, ans = 11503 - 10815...",[ 6.0 billion],6.0 billion,"Read the following text and table, and then an..."
...,...,...,...,...,...,...,...,...,...,...,...
153,"[between the years of 2017 and 2018, what was ...",0.0,table of contents adobe inc . notes to consoli...,- | 2018 | 2017\nbeginning balance | $ 172945 ...,to the extent that the foreign earnings previo...,- | 2018 | 2017\nending balance | $ 196152 | $...,Double_ADBE/2018/page_86.pdf,"[ans = 196152 - 172945, ans = 172945, x0 = 196...",[\n#Python\ntotal_gross_amount_unrecognized_ta...,-4.7911,"Read the following text and table, and then an..."
154,[what was the lease payments to the employee w...,2.8,the future minimum lease commitments under the...,2011 | $ 62465\n2012 | 54236\n2013 | 47860\n20...,lease payments to an employee who became an of...,,Single_LKQ/2010/page_84.pdf-4,"[ans = 0.9, ans = 0.9, ans = 0.9 + 0.9, x0 = 0...",[ what is the total payment for all three year...,what is the total payment for all three years...,"Read the following text and table, and then an..."
155,[how much does the total of intangibles repres...,80.0,devon energy corporation and subsidiaries note...,year | amortization amount ( in millions )\n20...,amortization expense for intangibles was appro...,year | amortization amount ( in millions )\n20...,Single_DVN/2014/page_85.pdf-2,"[ans = 36 / 45, x0 = 36 / 45; ans = x0 * 100]",[\n#Python\namortization_expense_2014 = 36\nam...,55.555556,"Read the following text and table, and then an..."
156,[what was the difference in rent payment after...,0.13793,used to refinance certain indebtedness which m...,year | amount\n2018 | 141\n2019 | 132\n2020 | ...,the lease requires annual base rental payments...,,Single_BLK/2017/page_122.pdf-3,"[ans = 66 - 58, ans = 58, x0 = 66 - 58; ans = ...",[\n#Python\nrent_payment_after_5_years = 